In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

## Taking text for summarization

from bs4 import BeautifulSoup as bs
import requests

url = "https://www.bbc.com/news/health-61553822"
response = requests.get(url)
#print(response)

html = response.content
#print(html)
soup = bs(html,"lxml")

import re
text = ",".join([el.text for el in soup.find_all("div", [re.compile('.*RichTextContainer.*')], recursive=True)])
#print(soup.find(property="articleBody"))
#article = [el.text for el in soup.find_all("div", [re.compile('.*RichTextContainer.*')], recursive= True)]
#print(text)


# List of stop words
stopwords = list( STOP_WORDS)
# pass document into spacy and store in "doc" object
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
tokens = [token.text for token in doc]
#print(tokens)

#List item
# add \n to the punchuvation list
punctuation = punctuation + '\n' + '\n\n'
punctuation

# Preparing a dictionary for word frequencies¶
word_frequencies = {}                       # Dictionary Name
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():   
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] +=1
#print(word_frequencies)
max_frequcncy = max(word_frequencies.values())

# Normalize the word frencies
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequcncy
     
#sentence tokenization -Calculate sentences scores¶   By creating a Dictionay for sentences and its normalized frequencies
sentence_tokens = [sent for sent in doc.sents]
#print(sentence_tokens)
sentence_scores = {}                                            #create Dictionary
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():        
            if sent not in sentence_scores.keys():              # add word normalized frequencies count in each of these sentences,then with maximum value we are going to find important sentence 
                sentence_scores[sent] = word_frequencies[word.text.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.text.lower()]
sentence_scores

# Can tweek percentage
# Get 30% of important sentences with maximum score - using nlargest
from heapq import nlargest
select_length = int(len(sentence_tokens)*0.1)
select_length
summary = nlargest(select_length , sentence_scores , key = sentence_scores.get)
summary   # Got 4 important sentences
final_summary = [word.text for word in summary]
final_summary
#print(len(final_summary))
#combining above 4 lines togeather to from a summary
summary = ' ' .join(final_summary)
summary
summary1 = summary.replace('\n', ' ')
print('\n')
print("Text summarization using SPACY")
print(summary1)
print('\n')
print("length of original text:",len(text))
print("length of autosummarization text using SPACY",len(summary))

# Summarization using 'gensim'
#from gensim.summarization import summarize
#print('\n')
#print("Text summarization using gensim")
#summarize(text)



Text summarization using SPACY
,A person is considered at high risk of having caught the infection if they have had household or sexual contact with someone with monkeypox, or have changed the bedding of an infected person without wearing personal protective equipment (PPE).,Symptoms, which include a high temperature, aches, and a rash of raised spots that later turn into blisters, are typically mild and for most people clear up within two to four weeks. ,"However the likelihood of further spread of the virus through close contact for example during sexual activities amongst persons with multiple sexual partners is considered to be high".


length of original text: 2810
length of autosummarization text using SPACY 615
